## 解析HTML内容

In [1]:
from lxml import etree
#输入一端HTML文本
text='''
<div>
<ul>
<li class="item-0"><a href="link1.html">first item</a></li>
<li class="item-1"><a href="link2.html">second item</a></li>
<li class="item-inactive"><a href="link3.html">third item</a></li>
<li class="item-1"><a href="link4.html">fourth item</a></li>
<li class="item-0"><a href="link5.html">fifth item</a>
</ul>
</div>
'''
#构造HTML解析器
html = etree.HTML(text)
#将解析器转化为字符串,输出为Bytes型
result = etree.tostring(html)
#utf-8格式输出
print(result.decode("utf-8"))
#我们发现，HTML文本被补齐<html><body>标签，以及缺失的fifthe item后的</li>标签

<html><body><div>
<ul>
<li class="item-0"><a href="link1.html">first item</a></li>
<li class="item-1"><a href="link2.html">second item</a></li>
<li class="item-inactive"><a href="link3.html">third item</a></li>
<li class="item-1"><a href="link4.html">fourth item</a></li>
<li class="item-0"><a href="link5.html">fifth item</a>
</li></ul>
</div>
</body></html>


## 读取文件解析HTML文本

In [2]:
#直接读取文本文件进行解析
from lxml import etree
html = etree.parse('./test.html',etree.HTMLParser())
result = etree.tostring(html)
print(result.decode('utf-8'))
#本次解析，多了一个DOCTYPE声明，不影响后续流程

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN" "http://www.w3.org/TR/REC-html40/loose.dtd">
<html><body><div>&#13;
<ul>&#13;
<li class="item-0"><a href="link1.html">first item</a></li>&#13;
<li class="item-1"><a href="link2.html">second item</a></li>&#13;
<li class="item-inactive"><a href="link3.html">third item</a></li>&#13;
<li class="item-1"><a href="link4.html">fourth item</a></li>&#13;
<li class="item-0"><a href="link5.html">fifth item</a>&#13;
</li></ul>&#13;
</div></body></html>


## 属性选择

In [3]:
from lxml import etree
html = etree.parse('./test.html',etree.HTMLParser())
#选取li节点下的class属性，值为item-0,返回为列表
result_attr = html.xpath('//li[@class="item-0"]')
print(result_attr)

[<Element li at 0x1c6d64d5fc8>, <Element li at 0x1c6d65ade48>]


## 选取子节点

In [4]:
from lxml import etree
html = etree.parse('./test.html',etree.HTMLParser())
#匹配所有节点
result_all_node = html.xpath('//*')
print(result_all_node)
#匹配所有li节点
result_all_node_li = html.xpath('//li')
print(result_all_node_li)
#匹配所有li节点下a节点，即获取子节点
result_all_node_li_a = html.xpath('//li/a')
print(result_all_node_li_a)
#匹配所有ul节点下a节点,即获取子孙节点
result_all_node_ul_a = html.xpath('//ul//a')
print(result_all_node_ul_a)
#如果使用/a，则无法获取到子孙节点
result_all_node_ul_except = html.xpath('//ul/a')
print(result_all_node_ul_except)

[<Element html at 0x1c6d65dc788>, <Element body at 0x1c6d65dc808>, <Element div at 0x1c6d65dc848>, <Element ul at 0x1c6d65dc888>, <Element li at 0x1c6d65dc8c8>, <Element a at 0x1c6d65dc948>, <Element li at 0x1c6d65dc988>, <Element a at 0x1c6d65dc9c8>, <Element li at 0x1c6d65dca08>, <Element a at 0x1c6d65dc908>, <Element li at 0x1c6d65dca48>, <Element a at 0x1c6d65dca88>, <Element li at 0x1c6d65dcac8>, <Element a at 0x1c6d65dcb08>]
[<Element li at 0x1c6d65dc8c8>, <Element li at 0x1c6d65dc988>, <Element li at 0x1c6d65dca08>, <Element li at 0x1c6d65dca48>, <Element li at 0x1c6d65dcac8>]
[<Element a at 0x1c6d65dc948>, <Element a at 0x1c6d65dc9c8>, <Element a at 0x1c6d65dc908>, <Element a at 0x1c6d65dca88>, <Element a at 0x1c6d65dcb08>]
[<Element a at 0x1c6d65dc948>, <Element a at 0x1c6d65dc9c8>, <Element a at 0x1c6d65dc908>, <Element a at 0x1c6d65dca88>, <Element a at 0x1c6d65dcb08>]
[]


## 选取父节点

In [5]:
from lxml import etree
html = etree.parse('./test.html',etree.HTMLParser())
#匹配子节点a，href属性值为"link4.html"的节点后，寻找..父节点的属性class
result_parent = html.xpath('//a[@href="link4.html"]/../@class')
print(result_parent)

['item-1']


## 文本获取

In [6]:
from lxml import etree
html = etree.parse('./test.html',etree.HTMLParser())
#匹配子节点a
result_a = html.xpath('//a[@href="link4.html"]')
#使用text属性获取文本
print(result_a[0].text)
#如果直接选取属性class="item-inactive"下调用text()，很明显无法调用成功
result_ax = html.xpath('//li[@class="item-inactive"]/text()')
print(result_ax)
#但是可以增加定位到子节点a
result_ax = html.xpath('//li[@class="item-1"]/a/text()')
print(result_ax)
#或者使用//,多出来的\r\n，是由于在解析html文件时，进行了自动修正，补充了尾部标签的换行。
result_ax = html.xpath('//li[@class="item-0"]//text()')
print(result_ax)

fourth item
[]
['second item', 'fourth item']
['first item', 'fifth item', '\r\n']


## 属性获取

In [7]:
#获取如href的值
from lxml import etree
html = etree.parse('./test.html',etree.HTMLParser())
#匹配子节点a下的属性href，与a[@href=value]属性匹配不同，此处由于进行属性获取，不加[]
result_attri = html.xpath('//a/@href')
#返回值为列表形式
print(result_attri)

['link1.html', 'link2.html', 'link3.html', 'link4.html', 'link5.html']


### 属性值模糊匹配
> 当属性值包含某个特征，可以使用contains(第一个参数属性名称,模糊匹配即可的属性值)

In [8]:
from lxml import etree
text = '''
<li class="li li-first"><a href="link.html">first item</a></li>
'''
html = etree.HTML(text)
result = html.xpath('//li[contains(@class,"li-f")]/a/text()')
print(result)

['first item']


### 多属性匹配
> 如果遇到节点中含有多个属性，那么可以使用and进行连接

In [9]:
from lxml import etree
text = '''
<li class="li li-first" name="item"><a href="link.html">first item</a></li>
'''
html = etree.HTML(text)
#注意，and需要在li节点下[]中使用
result = html.xpath('//li[contains(@class,"li") and @name="item"]/a/text()')
print(result)

['first item']


## XPath 运算符


下面列出了可用在 XPath 表达式中的运算符：

| 运算符 | 描述           | 实例                      | 返回值                                                       |
| ------ | -------------- | ------------------------- | ------------------------------------------------------------ |
| 丨   | 计算两个节点集 | //book 丨 //cd            | 返回所有拥有 book 和 cd 元素的节点集                         |
| +      | 加法           | 6 + 4                     | 10                                                           |
| -      | 减法           | 6 - 4                     | 2                                                            |
| *      | 乘法           | 6 * 4                     | 24                                                           |
| div    | 除法           | 8 div 4                   | 2                                                            |
| =      | 等于           | price=9.80                | 如果 price 是 9.80，则返回 true。如果 price 是 9.90，则返回 false。 |
| !=     | 不等于         | price!=9.80               | 如果 price 是 9.90，则返回 true。如果 price 是 9.80，则返回 false。 |
| <      | 小于           | price<9.80                | 如果 price 是 9.00，则返回 true。如果 price 是 9.90，则返回 false。 |
| <=     | 小于或等于     | price<=9.80               | 如果 price 是 9.00，则返回 true。如果 price 是 9.90，则返回 false。 |
| >      | 大于           | price>9.80                | 如果 price 是 9.90，则返回 true。如果 price 是 9.80，则返回 false。 |
| >=     | 大于或等于     | price>=9.80               | 如果 price 是 9.90，则返回 true。如果 price 是 9.70，则返回 false。 |
| or     | 或             | price=9.80 or price=9.70  | 如果 price 是 9.80，则返回 true。如果 price 是 9.50，则返回 false。 |
| and    | 与             | price>9.00 and price<9.90 | 如果 price 是 9.80，则返回 true。如果 price 是 8.50，则返回 false。 |
| mod    | 计算除法的余数 | 5 mod 2                   | 1                                                            |

## 按序选择
> 当配配多个节点的时候，可以通过传入索引，选择特定的次序。

In [10]:
from lxml import etree
html = etree.parse('./test.html',etree.HTMLParser())
#选择索引第一位
result_1 = html.xpath('//li[1]/a/text()')
print(result_1)
#选择索引最后一位
result_2 = html.xpath('//li[last()]/a/text()')
print(result_2)
#选择索引第3位前
result_3 = html.xpath('//li[position()<3]/a/text()')
print(result_3)
#选择索引倒数第3位
result_4 = html.xpath('//li[last()-2]/a/text()')
print(result_4)

['first item']
['fifth item']
['first item', 'second item']
['third item']


## 节点轴选择
> Xpath提供了很多节点轴的选择方法，包括获取子元素、兄弟元素、父元素、祖先元素等。

In [11]:
from lxml import etree
html = etree.parse('./test.html',etree.HTMLParser())
#调用ancestor后去所有祖先节点，后续跟::，然后加节点选择器，使用*表示匹配所有节点，因此，返回了第一个li及诶单的所有祖先节点，li>ul>div>body>html
result = html.xpath('//li[1]/ancestor::*')
print(result)
#限定祖先节点仅div
result = html.xpath('//li[1]/ancestor::div')
print(result)
#调用ancestor选择所有的属性值，使用*代表获取节点的所有属性，及li节点的所有属性值
result = html.xpath('//li[1]/attribute::*')
print(result)
#调用child，获取所有直接的子节点，然后限定为href="link1.html"的a节点
result = html.xpath('//li[1]/child::a[@href="link1.html"]')
print(result)
#调用descendant，获取所有子孙节点，并限定节点为span，因为没有span，所以未输出
result = html.xpath('//li[1]/descendant::span')
print(result)
#调用following，直接获取当前节点后所有节点，通过*匹配，但是增加索引，故获取的为第二个后续节点
result = html.xpath('//li[1]/following::*[2]')
print(result)
#调用following-sibling获取当前节点的所有同级节点，指定*匹配，即所有同级节点
result = html.xpath('//li[1]/following-sibling::*')
print(result)

[<Element html at 0x1c6d65e7688>, <Element body at 0x1c6d65e7788>, <Element div at 0x1c6d65e77c8>, <Element ul at 0x1c6d65e7808>]
[<Element div at 0x1c6d65e77c8>]
['item-0']
[<Element a at 0x1c6d65e79c8>]
[]
[<Element a at 0x1c6d65e77c8>]
[<Element li at 0x1c6d65e7788>, <Element li at 0x1c6d65e7a08>, <Element li at 0x1c6d65e7a48>, <Element li at 0x1c6d65e7a88>]
